# 1. 개념 문제

1-1. SentiSynset 객체는 ( 긍정감성 )지수와 ( 부정감성 )지수를 가지고 있다. 

1-2. ( )는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon이다. 이를 이용하면 먼저 SentimentIntensityAnalyzer 객체를 생성한 뒤에 문서별로 ( )메서드를 호출해 감성 점수를 구한 뒤, 해당 문서의 감성 점수가 특정 임계값 이상이면( ), 그렇지 않으면 ( )으로 판단한다.

1-3. 텍스트 분류 기반의 문서 분류와 달리 문서 군집화는 학습 데이터 세트가 필요없는 (     ) 으로 동작한다.

1-4. 사이킷런의 KMenas 객체는 각 군집을 구성하는 단어 피처가 군집의 중심 ( ) 을 기준으로 얼마나 가깝게 위치해 있는지 ( ) 라는 속성으로 제공한다

1-5. (  ) 란 최상위 루트로부터 경유한 경로를 전부 기입하는 방식이며 어떤 파일이 가진 고유한 경로를 뜻한다.

# 2. 감성분석

2-1. 주어진 텍스트를 아래와 같이 변환하기 위해 빈칸을 채워주세요

In [1]:
import pandas as pd
review_df = pd.read_csv('./labeledTrainData.tsv', header=0, sep="\t", quoting=3)

In [2]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [3]:
import re
review_df['review'] = review_df['review'].str.replace('<br />',' ')

review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

In [4]:
print(review_df['review'][0])

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

2-2. 'alarm'의 긍정감성지수, 부정감성지수, 객관성지수를 확인해봅시다

In [6]:
import nltk
from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')

alarm = swn.senti_synset('alarm.n.01')

print('alarm 긍정감성 지수: ', alarm.pos_score())
print('alarm 부정감성 지수: ', alarm.neg_score())
print('alarm 객관성 지수: ', alarm.obj_score())

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\jaeyong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\sentiwordnet.zip.


alarm 긍정감성 지수:  0.375
alarm 부정감성 지수:  0.25
alarm 객관성 지수:  0.375


alarm 긍정감성 지수: 0.375

alarm 부정감성 지수: 0.25

alarm 객관성 지수: 0.375

2-3. 빈칸을 채우며 SentiWordNet을 이용한 감성분석과 VADER를 이용한 감성분석 과정을 복습해봅시다

In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return

def swn_polarity(text):
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1          
    
    if not tokens_count:
        return 0
    
    if sentiment >= 0 :
        return 1
    
    return 0

review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
review_df.head(10)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaeyong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


,id,sentiment,review,preds
0,"""5814_8""",1,With all this stuff going down at the moment ...,0
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...,1
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...,0
3,"""3630_4""",0,It must be assumed that those who praised thi...,0
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...,0
5,"""8196_8""",1,I dont know why people think this is such a b...,1
6,"""7166_2""",0,This movie could have been very good but com...,0
7,"""10633_1""",0,I watched this video at a friend s house I m...,0
8,"""319_1""",0,A friend of mine bought this film for and...,0
9,"""8713_10""",1,This movie is full of references Like Ma...,1


In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def vader_polarity(review,threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )
review_df.head(10)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jaeyong\AppData\Roaming\nltk_data...


,id,sentiment,review,preds,vader_preds
0,"""5814_8""",1,With all this stuff going down at the moment ...,0,0
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...,1,1
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...,0,0
3,"""3630_4""",0,It must be assumed that those who praised thi...,0,1
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...,0,1
5,"""8196_8""",1,I dont know why people think this is such a b...,1,1
6,"""7166_2""",0,This movie could have been very good but com...,0,0
7,"""10633_1""",0,I watched this video at a friend s house I m...,0,0
8,"""319_1""",0,A friend of mine bought this film for and...,0,1
9,"""8713_10""",1,This movie is full of references Like Ma...,1,1


In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test=None, pred=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [15]:
y1_target = review_df['sentiment'].values
preds = review_df['preds'].values
print('#### SentiWordNet 예측 성능 평가 ####')
get_clf_eval(y1_target, preds)

y2_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values
print('#### VADER 예측 성능 평가 ####')
get_clf_eval(y2_target, vader_preds)

#### SentiWordNet 예측 성능 평가 ####
오차 행렬
[[7668 4832]
 [3636 8864]]
정확도: 0.6613, 정밀도: 0.6472, 재현율: 0.7091,    F1: 0.6767, AUC:0.6613
#### VADER 예측 성능 평가 ####
오차 행렬
[[ 6736  5764]
 [ 1867 10633]]
정확도: 0.6948, 정밀도: 0.6485, 재현율: 0.8506,    F1: 0.7359, AUC:0.6948


# 3. 간단한 토픽 모델링

주어진 짧은 sentence를 가지고 LDA를 이용해 간단한 토픽 모델링을 진행해봅시다.

In [30]:
import nltk
from nltk import word_tokenize, sent_tokenize

In [31]:
sentence = 'In the 200th episode of The Life Scientific, Jim Al-Khalili finds out why Demis Hassabis wants to create artificial intelligence and use it to help humanity. Thinking about how to win at chess when he was a boy got Demis thinking about the process of thinking itself. Being able to program his first computer (a Sinclair Spectrum) felt miraculous. In computer chess, his two passions were combined. And a lifelong ambition to create artificial intelligence was born. Demis studied computer science at Cambridge and then worked in the computer games industry for many years. Games, he says, are the ideal testing ground for AI. Then, thinking memory and imagination were aspects of the human mind that would be a necessary part of any artificially intelligent system, he studied neuroscience for a PhD. He set up DeepMind in 2010 and pioneered a new approach to creating artificial intelligence, based on deep learning and built-in rewards for making good decisions. Four years later, DeepMind was sold to Google for £400 million. The company’s landmark creation, Alpha Go stunned the world when it defeated the world Go champion in South Korea in 2016. Their AI system, AlphaZero taught itself to play chess from scratch. After playing against itself for just four hours, it was the best chess computer in the world. (Humans had been defeated long ago). Many fear both the supreme intelligence and the stupidity of AI. Demis imagines a future in which computers and humans put their brains together to try and understand the world. His algorithms have inspired humans to raise their game, when playing Go and chess. Now, he hopes that AI might do the same for scientific research. Perhaps the next Nobel Prize will be shared between a human and AI? Producer: Anna Buckley'

### 3-1. #___________ 를 채워 넣어 주세요.

In [32]:
def tokenizer(text):
    sentences = sent_tokenize(text)
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

In [33]:
word_tokens = tokenizer(sentence)
print(word_tokens)

[['In', 'the', '200th', 'episode', 'of', 'The', 'Life', 'Scientific', ',', 'Jim', 'Al-Khalili', 'finds', 'out', 'why', 'Demis', 'Hassabis', 'wants', 'to', 'create', 'artificial', 'intelligence', 'and', 'use', 'it', 'to', 'help', 'humanity', '.'], ['Thinking', 'about', 'how', 'to', 'win', 'at', 'chess', 'when', 'he', 'was', 'a', 'boy', 'got', 'Demis', 'thinking', 'about', 'the', 'process', 'of', 'thinking', 'itself', '.'], ['Being', 'able', 'to', 'program', 'his', 'first', 'computer', '(', 'a', 'Sinclair', 'Spectrum', ')', 'felt', 'miraculous', '.'], ['In', 'computer', 'chess', ',', 'his', 'two', 'passions', 'were', 'combined', '.'], ['And', 'a', 'lifelong', 'ambition', 'to', 'create', 'artificial', 'intelligence', 'was', 'born', '.'], ['Demis', 'studied', 'computer', 'science', 'at', 'Cambridge', 'and', 'then', 'worked', 'in', 'the', 'computer', 'games', 'industry', 'for', 'many', 'years', '.'], ['Games', ',', 'he', 'says', ',', 'are', 'the', 'ideal', 'testing', 'ground', 'for', 'AI', 

### 3-2. print 값을 참고해서 stop word를 제거하고 1차원 filtered_words 리스트를 완성하세요.

In [34]:
stopwords = nltk.corpus.stopwords.words('english')
filtered_words  = []

In [35]:
for sentence in word_tokens:
    for word in sentence:
        word = word.lower()
        if word not in stopwords:
            filtered_words.append(word)

In [36]:
print(filtered_words)

['200th', 'episode', 'life', 'scientific', ',', 'jim', 'al-khalili', 'finds', 'demis', 'hassabis', 'wants', 'create', 'artificial', 'intelligence', 'use', 'help', 'humanity', '.', 'thinking', 'win', 'chess', 'boy', 'got', 'demis', 'thinking', 'process', 'thinking', '.', 'able', 'program', 'first', 'computer', '(', 'sinclair', 'spectrum', ')', 'felt', 'miraculous', '.', 'computer', 'chess', ',', 'two', 'passions', 'combined', '.', 'lifelong', 'ambition', 'create', 'artificial', 'intelligence', 'born', '.', 'demis', 'studied', 'computer', 'science', 'cambridge', 'worked', 'computer', 'games', 'industry', 'many', 'years', '.', 'games', ',', 'says', ',', 'ideal', 'testing', 'ground', 'ai', '.', ',', 'thinking', 'memory', 'imagination', 'aspects', 'human', 'mind', 'would', 'necessary', 'part', 'artificially', 'intelligent', 'system', ',', 'studied', 'neuroscience', 'phd', '.', 'set', 'deepmind', '2010', 'pioneered', 'new', 'approach', 'creating', 'artificial', 'intelligence', ',', 'based', 

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

count_vect = CountVectorizer()
feat_vect = count_vect.fit_transform(filtered_words)
lda = LatentDirichletAllocation(n_components = 1, random_state = 10)
lda.fit(feat_vect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=1, n_jobs=None,
                          perp_tol=0.1, random_state=10, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

### 3-3. stop word를 제거한 후 남은 word 피처의 개수는 몇 개인가요?

In [43]:
len(filtered_words)

211

In [44]:
# 정답 : 211

### 3-4. display_topics 함수를 완성하여 연관도가 높은 순으로 Word를 나열하는 함수를 만들어주세요.

In [45]:
def display_topics(model, features_names, no_top_words):
     for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

In [48]:
feature_names = count_vect.get_feature_names()
display_topics(lda, feature_names, 10)

Topic # 0
ai chess computer intelligence world demis thinking artificial humans go


### 3-5. 출력 결과 단어 10개를 적어주세요.

In [49]:
# 10개 word : ai chess computer intelligence world demis thinking artificial humans go